# SMC & SMC'

In [1]:
# Import packages.
import copy
import matplotlib
from matplotlib import pyplot as plt
import msprime
import numpy as np
import pandas as pd
import smclib
# Print version numbers.
print('matplotlib', matplotlib.__version__)
print('msprime', msprime.__version__)
print('numpy', np.__version__)
print('pandas', pd.__version__)
# Intialize the matplolib styling.
plt.rcParams.update({
    'figure.constrained_layout.use': True,
    'figure.facecolor': 'white',
    'axes.spines.top': False,
    'axes.spines.right': False,
    'legend.frameon': False,
})

matplotlib 3.5.3
msprime 1.2.0
numpy 1.26.0
pandas 1.5.3


## Generate Simulated Data

Theoretically, you only need to do this once...

### Number of Recombination Events

In [2]:
# Intialize an lisy of populuation recombination rate values.
R_rhos = np.array([0.1, 1, 10, 100, 1_000])
# Intialize a dictionary to store the receombination event results.
recomb_dicc = {
    'standard':       {},
    'msp_smc':        {},
    'djp_smc':        {},
    'msp_smc_prime':  {},
    'djp_smc_prime':  {},
}
# For every population recombination rate.
for i, rho in enumerate(R_rhos):
    # Intialize lists to store the results.
    recomb_dicc['standard'][rho]      = []
    recomb_dicc['msp_smc'][rho]       = []
    recomb_dicc['djp_smc'][rho]       = []
    recomb_dicc['msp_smc_prime'][rho] = []
    recomb_dicc['djp_smc_prime'][rho] = []
    # Assuming Ne = 1, compute r.
    r = rho / 4
    # For 1000 replicate simulations.
    for _ in range(1000):
        # Simulate tree-sequences.
        ts_standard      = msprime.sim_ancestry(
            samples=[msprime.SampleSet(2, ploidy=1)],
            population_size=1,
            ploidy=2,
            recombination_rate=r,
            sequence_length=1,
            record_full_arg=True,
            discrete_genome=False,
        )
        ts_msp_smc       = msprime.sim_ancestry(
            samples=[msprime.SampleSet(2, ploidy=1)],
            population_size=1,
            ploidy=2,
            recombination_rate=r,
            sequence_length=1,
            model='smc',
            record_full_arg=True,
            discrete_genome=False,
        )
        ts_msp_smc_prime = msprime.sim_ancestry(
            samples=[msprime.SampleSet(2, ploidy=1)],
            population_size=1,
            ploidy=2,
            recombination_rate=r,
            sequence_length=1,
            model='smc_prime',
            record_full_arg=True,
            discrete_genome=False,
        )
        ts_djp_smc       = smclib.sim_smc(k=2, Ne=1, rho=rho, ploidy=2)
        ts_djp_smc_prime = smclib.sim_smc_prime(k=2, Ne=1, rho=rho, ploidy=2)
        # Update the dictionary.
        recomb_dicc['standard'][rho].append(smclib.R_g_arg(ts_standard))
        recomb_dicc['msp_smc'][rho].append(smclib.R_g_arg(ts_msp_smc))
        recomb_dicc['djp_smc'][rho].append(smclib.R_arg_dicc(ts_djp_smc))
        recomb_dicc['msp_smc_prime'][rho].append(smclib.R_g_arg(ts_msp_smc_prime))
        recomb_dicc['djp_smc_prime'][rho].append(smclib.R_arg_dicc(ts_djp_smc_prime))

In [4]:
# For every model.
for model in ['standard', 'msp_smc', 'djp_smc', 'msp_smc_prime', 'djp_smc_prime']:
    # For every population recombination rate.
    for rho in R_rhos:
        # If the population recombination rate is 1, 10, or 100.
        if rho >= 1:
            # Format as an integer.
            rho = int(rho)
        # Export the number of recombination events result.
        np.savetxt(
            f'./data/{model}_rho_{rho}_num_recomb_events.txt.gz',
            [np.array(recomb_dicc[model][rho])], fmt='%d',
        )

### $i^{th}$ Tree Height

In [5]:
# Define a list of tree indicies.
tree_idx = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, -1])
# Intialize an lisy of populuation recombination rate values.
ith_rhos = np.array([1, 100])
# Intialize sample sizes.
ns = np.array([2, 3, 5, 10, 20])
# Intialize a dictionary to store the tree heights and width.
height_dicc = {
    'standard':      {},
    'msp_smc':       {},
    'djp_smc':       {},
    'msp_smc_prime': {},
    'djp_smc_prime': {},
}
width_dicc = {
    'standard':      {},
    'msp_smc':       {},
    'djp_smc':       {},
    'msp_smc_prime': {},
    'djp_smc_prime': {},
}
# For every sample size.
for n in ns:
    # Intialize subdictionaries.
    for key in height_dicc:
        height_dicc[key][n] = {}
        width_dicc[key][n]  = {}
    # For every population recombination rate.
    for rho in ith_rhos:
        # Intialize subdictionaries.
        for key in height_dicc:
            height_dicc[key][n][rho] = np.zeros((1_000, tree_idx.size))
            width_dicc[key][n][rho]  = np.zeros((1_000, tree_idx.size))
        # Assuming Ne = 1, compute r.
        r = rho / 4
        # For 1000 replicate simulations.
        for i in range(1000):
            # Simulate tree-sequences.
            ts_standard      = msprime.sim_ancestry(
                samples=[msprime.SampleSet(n, ploidy=1)],
                population_size=1,
                ploidy=2,
                recombination_rate=r,
                sequence_length=1,
                discrete_genome=False,
            )
            ts_msp_smc       = msprime.sim_ancestry(
                samples=[msprime.SampleSet(n, ploidy=1)],
                population_size=1,
                ploidy=2,
                recombination_rate=r,
                sequence_length=1,
                model='smc',
                discrete_genome=False,
            )
            ts_msp_smc_prime = msprime.sim_ancestry(
                samples=[msprime.SampleSet(n, ploidy=1)],
                population_size=1,
                ploidy=2,
                recombination_rate=r,
                sequence_length=1,
                model='smc_prime',
                discrete_genome=False,
            )
            ts_djp_smc       = smclib.ith_tree_smc(k=n, Ne=1, rho=rho, ploidy=2)
            ts_djp_smc_prime = smclib.ith_tree_smc_prime(k=n, Ne=1, rho=rho, ploidy=2)
            # For every ith tree index.
            for j in tree_idx:
                # Find the ith tree heights and widths.
                standard_height, standard_width           = smclib.ith_tmrca_ts(ts=ts_standard, ith_tree=j)
                msp_smc_height, msp_smc_width             = smclib.ith_tmrca_ts(ts=ts_msp_smc, ith_tree=j)
                djp_smc_height, djp_smc_width             = smclib.ith_tmrca_ts_dicc(ts_dicc=ts_djp_smc, ith_tree=j)
                msp_smc_prime_height, msp_smc_prime_width = smclib.ith_tmrca_ts(ts=ts_msp_smc_prime, ith_tree=j)
                djp_smc_prime_height, djp_smc_prime_width = smclib.ith_tmrca_ts_dicc(ts_dicc=ts_djp_smc_prime, ith_tree=j)
                # Update the results.
                height_dicc['standard'][n][rho][i, j]      = standard_height
                height_dicc['msp_smc'][n][rho][i, j]       = msp_smc_height
                height_dicc['djp_smc'][n][rho][i, j]       = djp_smc_height
                height_dicc['msp_smc_prime'][n][rho][i, j] = msp_smc_prime_height
                height_dicc['djp_smc_prime'][n][rho][i, j] = djp_smc_prime_height
                width_dicc['standard'][n][rho][i, j]       = standard_width
                width_dicc['msp_smc'][n][rho][i, j]        = msp_smc_width
                width_dicc['djp_smc'][n][rho][i, j]        = djp_smc_width
                width_dicc['msp_smc_prime'][n][rho][i, j]  = msp_smc_prime_width
                width_dicc['djp_smc_prime'][n][rho][i, j]  = djp_smc_prime_width

In [6]:
# For every model.
for model in ['standard', 'msp_smc', 'djp_smc', 'msp_smc_prime', 'djp_smc_prime']:
    # For every sample size.
    for n in ns:
        # For every population recombination rate.
        for rho in ith_rhos:
            # Export the tree height and width results.
            np.savetxt(
                f'./data/{model}_tips_{n}_rho_{rho}_ith_tree_height.txt.gz',
                height_dicc[model][n][rho], fmt='%1.15f',
            )
            np.savetxt(
                f'./data/{model}_tips_{n}_rho_{rho}_ith_tree_width.txt.gz',
                width_dicc[model][n][rho], fmt='%1.15f',
            )

## Validating Previous Results

Before validating I will intialize a convience dictionary.

In [7]:
# Define a model conversion dictionary.
model_dicc = {
    'standard':      'msprime Standard Coalescent',
    'msp_smc':       'msprime\'s SMC',
    'djp_smc':       'Dave\'s SMC',
    'msp_smc_prime': 'msprime\'s SMC\'',
    'djp_smc_prime': 'Dave\'s SMC\'',
}

### Number of Recombination Events

In [8]:
# Intialize an lisy of populuation recombination rate values.
R_rhos = np.array([0.1, 1, 10, 100, 1_000])
# Intialize a dictionary to store the results.
R_dicc = {}
# For every model.
for model in model_dicc:
    # Intialize the subdictionary.
    R_dicc[model] = {}
    # For every population recombination rate.
    for rho in R_rhos:
        # If the population recombination rate is 1, 10, or 100.
        if rho >= 1:
            # Fill the dictionary.
            R_dicc[model][rho] = np.loadtxt(f'./data/{model}_rho_{int(rho)}_num_recomb_events.txt.gz', dtype=int)
        # Else.
        else:
            # Fill the dictionary.
            R_dicc[model][rho] = np.loadtxt(f'./data/{model}_rho_{rho}_num_recomb_events.txt.gz', dtype=int)

In [9]:
# Intialize a dataframe dictionary.
df_R_mean = {
    'msprime Standard Coalescent': [],
    'msprime\'s SMC': [],
    'msprime\'s SMC\'': [],
    'Dave\'s SMC': [],
    'Dave\'s SMC\'': [],
}
# For every population recombination rate.
for rho in R_rhos:
    # For every model.
    for model in model_dicc:
        # Update the dataframe dictionary.
        df_R_mean[model_dicc[model]].append(np.mean(R_dicc[model][rho]))
# Convert the dictionary to a dataframe.
R_mean_df = pd.DataFrame(
    data=df_R_mean,
    index=[f'$\\rho = {int(rho)}$' if rho.is_integer() else f'$\\rho = {rho}$' for rho in R_rhos],
)

In [10]:
# Show the mean number of recombination events results (E[R] = rho).
R_mean_df

msprime Standard Coalescent  msprime's SMC  msprime's SMC'  \
$\rho = 0.1$                         0.123          0.118           0.109   
$\rho = 1$                           1.531          0.985           1.331   
$\rho = 10$                         19.673         11.440          13.668   
$\rho = 100$                       557.077        113.662         144.126   
$\rho = 1000$                    26282.907       1143.656        1442.091   

               Dave's SMC  Dave's SMC'  
$\rho = 0.1$        0.106        0.097  
$\rho = 1$          1.018        0.962  
$\rho = 10$        10.228        9.815  
$\rho = 100$       99.111      100.664  
$\rho = 1000$    1005.608     1004.483

In [11]:
# Intialize figures and axes.
fig, axes = plt.subplots(
     2, 2, figsize=(6, 4),
    sharex=False, sharey=False, dpi=300,
)
# Intialize a dictionary for plotting.
plot_dicc = {
    'msp_smc': {'title': 'msprime\'s SMC', 'y_lab': r'ratio $R_{CO} / R_{SMC}$'},
    'msp_smc_prime': {'title': 'msprime\'s SMC\'', 'y_lab': r'''ratio $R_{CO} / R_{SMC'}$'''},
    'djp_smc': {'title': 'Dave\'s SMC', 'y_lab': r'ratio $R_{CO} / R_{SMC}$'},
    'djp_smc_prime': {'title': 'Dave\'s SMC\'', 'y_lab': r'''ratio $R_{CO} / R_{SMC'}$'''},
}
# Intialize axes list.
ax_list = [
    (0, 0), (0, 1),
    (1, 0), (1, 1),
]
# Intialize the mean number of crossovers for the standard coalescent.
standard_mean_co = np.array([np.mean(R_dicc['standard'][rho]) for rho in R_rhos])
# For every key.
for i, key in enumerate(plot_dicc):
    # Unpack the row and column indicies.
    row, col = ax_list[i]
    # Compute the R ratio.
    R_ratio = standard_mean_co / np.array([np.mean(R_dicc[key][rho]) for rho in R_rhos])
    # Plot the the results.
    axes[row, col].scatter(
        R_rhos, R_ratio, color='black',
        marker='D', s=10,
    )
    # Set the subplot title.
    axes[row, col].set_title(plot_dicc[key]['title'], fontweight='bold', size=8)
    # Set the axes labels.
    axes[row, col].set_xlabel(r'$\rho$', size=7)
    axes[row, col].set_ylabel(plot_dicc[key]['y_lab'], size=7)
    # Move the left spine (y-axis bar) to x = 1.
    axes[row, col].spines['left'].set_position(('data', 1))
    # Set the x-axis scale.
    axes[row, col].set_xscale('log')
    # Set the y-axis ticks and tick labels.
    axes[row, col].set_yticks(np.round(R_ratio))
    axes[row, col].set_yticklabels([str(round(ratio)) for ratio in R_ratio], size=6)
    # Set the axes scale, ticks, and tick labels.
    axes[row, col].set_xticks(R_rhos)
    axes[row, col].set_xticklabels([str(int(rho)) if rho.is_integer() else str(rho) for rho in R_rhos], size=6)
# Show the plot.
plt.show()

### $i^{th}$ Tree Height

__Note:__ the $i^{th}$ tree height results from `msprime` are in units of generations while my results are in coalescent units.

In [12]:
# Define a list of tree indicies.
tree_idx = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, -1])
# Intialize an lisy of populuation recombination rate values.
ith_rhos = np.array([1, 100])
# Intialize sample sizes.
ns = np.array([2, 3, 5, 10, 20])
# Intialize dictionaries to store the results.
height_dicc = {}
width_dicc  = {}
# For every model.
for model in model_dicc:
    # Intialize the subdictionaries.
    height_dicc[model] = {}
    width_dicc[model]  = {}
    # For every sample size.
    for n in ns:
        # Intialize the subdictionaries.
        height_dicc[model][n] = {}
        width_dicc[model][n]  = {}
        # For every population recombination rate.
        for rho in ith_rhos:
            # Load the ith tree height and width results.
            height_dicc[model][n][rho] = np.loadtxt(f'./data/{model}_tips_{n}_rho_{rho}_ith_tree_height.txt.gz')
            width_dicc[model][n][rho]  = np.loadtxt(f'./data/{model}_tips_{n}_rho_{rho}_ith_tree_width.txt.gz')

In [13]:
# Intialize a dataframe dictionaries.
df_height = {
    r'Sampled Lineages $(n)$': [],
    r'Population Recombination Rate $(\rho)$': [],
    r'$i^{th}$ Tree': [],
    'msprime Standard Coalescent': [],
    'msprime\'s SMC': [],
    'msprime\'s SMC\'': [],
    'Dave\'s SMC': [],
    'Dave\'s SMC\'': [],
}
df_width = {
    r'Sampled Lineages $(n)$': [],
    r'Population Recombination Rate $(\rho)$': [],
    r'$i^{th}$ Tree': [],
    'msprime Standard Coalescent': [],
    'msprime\'s SMC': [],
    'msprime\'s SMC\'': [],
    'Dave\'s SMC': [],
    'Dave\'s SMC\'': [],
}
# For every sample size.
for n in ns:
    # For every population recombination rate.
    for rho in ith_rhos:
        # For every tree index.
        for idx in tree_idx:
            # If the ith tree is not the last tree index.
            if idx != -1:
                # Add one to account for 0-indexing.
                ith = int(idx + 1)
            # Else.
            else:
                # Denote that this is the last tree.
                ith = 'last tree'
            # Update the dataframe dictionaries
            df_height[r'Sampled Lineages $(n)$'].append(n)
            df_height[r'Population Recombination Rate $(\rho)$'].append(rho)
            df_height[r'$i^{th}$ Tree'].append(ith)
            df_width[r'Sampled Lineages $(n)$'].append(n)
            df_width[r'Population Recombination Rate $(\rho)$'].append(rho)
            df_width[r'$i^{th}$ Tree'].append(ith)
            for model in model_dicc:
                # Update the dataframe dictionaries.
                df_height[model_dicc[model]].append(np.nanmean(height_dicc[model][n][rho][:, idx]))
                df_width[model_dicc[model]].append(np.nanmean(width_dicc[model][n][rho][:, idx]))
# Convert the dictionaries to data frames.
height_df = pd.DataFrame(data=df_height)
width_df = pd.DataFrame(data=df_width)

/var/folders/nr/v5td4f0d4c52kd3_ynmvd7r40000gn/T/ipykernel_57296/2509969906.py:45: RuntimeWarning: Mean of empty slice
  df_height[model_dicc[model]].append(np.nanmean(height_dicc[model][n][rho][:, idx]))
/var/folders/nr/v5td4f0d4c52kd3_ynmvd7r40000gn/T/ipykernel_57296/2509969906.py:46: RuntimeWarning: Mean of empty slice
  df_width[model_dicc[model]].append(np.nanmean(width_dicc[model][n][rho][:, idx]))


In [14]:
# Show the ith tree height results for 2 tips (E[tMRCA_2] = 1).
height_df[height_df[r'Sampled Lineages $(n)$'] == 2].set_index([
    r'Sampled Lineages $(n)$',
    r'Population Recombination Rate $(\rho)$',
    r'$i^{th}$ Tree',
])

msprime Standard Coalescent  \
Sampled Lineages $(n)$ Population Recombination Rate $(\rho)$ $i^{th}$ Tree                                
2                      1                                      1                                 1.999289   
                                                              2                                 3.277737   
                                                              3                                 4.163098   
                                                              4                                 5.148559   
                                                              5                                 4.960747   
                                                              6                                 3.118306   
                                                              7                                      NaN   
                                                              8                                      NaN   
                                                              9                                      NaN   
                                                              10                                     NaN   
                                                              last tree                         2.065704   
                       100                                    1                                 1.986066   
                                                              2                                 2.902371   
                                                              3                                 3.243863   
                                                              4                                 3.423862   
                                                              5                                 3.704755   
                                                              6                                 3.668073   
                                                              7                                 3.710856   
                                                              8                                 3.679808   
                                                              9                                 3.738678   
                                                              10                                3.612116   
                                                              last tree                         1.889012   

                                                                             msprime's SMC  \
Sampled Lineages $(n)$ Population Recombination Rate $(\rho)$ $i^{th}$ Tree                  
2                      1                                      1                   1.918462   
                                                              2                   3.554404   
                                                              3                   4.491346   
                                                              4                   4.915495   
                                                              5                   4.036110   
                                                              6                   5.733044   
                                                              7                   7.142423   
                                                              8                   7.796790   
                                                              9                        NaN   
                                                              10                       NaN   
                                                              last tree           1.948583   
                       100                                    1                   2.077536   
                                                              2                   2.931734   
                                                              3           

In [15]:
# Show the ith tree height results for 3 tips (E[tMRCA_3] = 1.333).
height_df[height_df[r'Sampled Lineages $(n)$'] == 3].set_index([
    r'Sampled Lineages $(n)$',
    r'Population Recombination Rate $(\rho)$',
    r'$i^{th}$ Tree',
])

msprime Standard Coalescent  \
Sampled Lineages $(n)$ Population Recombination Rate $(\rho)$ $i^{th}$ Tree                                
3                      1                                      1                                 2.656198   
                                                              2                                 3.514707   
                                                              3                                 4.293051   
                                                              4                                 4.531257   
                                                              5                                 4.867742   
                                                              6                                 3.286721   
                                                              7                                 4.642364   
                                                              8                                 3.094908   
                                                              9                                      NaN   
                                                              10                                     NaN   
                                                              last tree                         2.678467   
                       100                                    1                                 2.700865   
                                                              2                                 3.228319   
                                                              3                                 3.474071   
                                                              4                                 3.594388   
                                                              5                                 3.891012   
                                                              6                                 3.893508   
                                                              7                                 3.982422   
                                                              8                                 3.922473   
                                                              9                                 3.766357   
                                                              10                                3.892429   
                                                              last tree                         2.523664   

                                                                             msprime's SMC  \
Sampled Lineages $(n)$ Population Recombination Rate $(\rho)$ $i^{th}$ Tree                  
3                      1                                      1                   2.627678   
                                                              2                   3.663987   
                                                              3                   4.275709   
                                                              4                   4.855890   
                                                              5                   4.817699   
                                                              6                   4.974336   
                                                              7                   5.192543   
                                                              8                   6.674991   
                                                              9                   3.556232   
                                                              10                  3.473030   
                                                              last tree           2.691776   
                       100                                    1                   2.618094   
                                                              2                   3.431482   
                                                              3           

In [16]:
# Show the ith tree height results for 5 tips (E[tMRCA_5] = 1.6).
height_df[height_df[r'Sampled Lineages $(n)$'] == 5].set_index([
    r'Sampled Lineages $(n)$',
    r'Population Recombination Rate $(\rho)$',
    r'$i^{th}$ Tree',
])

msprime Standard Coalescent  \
Sampled Lineages $(n)$ Population Recombination Rate $(\rho)$ $i^{th}$ Tree                                
5                      1                                      1                                 3.295748   
                                                              2                                 3.767480   
                                                              3                                 4.254073   
                                                              4                                 4.429866   
                                                              5                                 4.561432   
                                                              6                                 4.837947   
                                                              7                                 5.984620   
                                                              8                                 4.401583   
                                                              9                                      NaN   
                                                              10                                     NaN   
                                                              last tree                         3.227648   
                       100                                    1                                 3.188720   
                                                              2                                 3.527565   
                                                              3                                 3.601981   
                                                              4                                 3.695871   
                                                              5                                 3.723355   
                                                              6                                 3.801841   
                                                              7                                 3.927203   
                                                              8                                 3.903906   
                                                              9                                 3.949616   
                                                              10                                3.958371   
                                                              last tree                         3.071671   

                                                                             msprime's SMC  \
Sampled Lineages $(n)$ Population Recombination Rate $(\rho)$ $i^{th}$ Tree                  
5                      1                                      1                   3.034403   
                                                              2                   3.689063   
                                                              3                   4.325551   
                                                              4                   4.653405   
                                                              5                   4.870284   
                                                              6                   5.335164   
                                                              7                   5.771129   
                                                              8                   6.328336   
                                                              9                   6.518933   
                                                              10                  4.521222   
                                                              last tree           3.173352   
                       100                                    1                   3.262621   
                                                              2                   3.517648   
                                                              3           

In [17]:
# Show the ith tree height results for 10 tips (E[tMRCA_10] = 1.8).
height_df[height_df[r'Sampled Lineages $(n)$'] == 10].set_index([
    r'Sampled Lineages $(n)$',
    r'Population Recombination Rate $(\rho)$',
    r'$i^{th}$ Tree',
])

msprime Standard Coalescent  \
Sampled Lineages $(n)$ Population Recombination Rate $(\rho)$ $i^{th}$ Tree                                
10                     1                                      1                                 3.567524   
                                                              2                                 3.799096   
                                                              3                                 4.178074   
                                                              4                                 4.600413   
                                                              5                                 4.670277   
                                                              6                                 4.945242   
                                                              7                                 5.476548   
                                                              8                                 5.374852   
                                                              9                                 5.491350   
                                                              10                                4.829326   
                                                              last tree                         3.564011   
                       100                                    1                                 3.619106   
                                                              2                                 3.824291   
                                                              3                                 3.853476   
                                                              4                                 3.921546   
                                                              5                                 4.031405   
                                                              6                                 4.075454   
                                                              7                                 4.144942   
                                                              8                                 4.246131   
                                                              9                                 4.181675   
                                                              10                                4.177976   
                                                              last tree                         3.630930   

                                                                             msprime's SMC  \
Sampled Lineages $(n)$ Population Recombination Rate $(\rho)$ $i^{th}$ Tree                  
10                     1                                      1                   3.542951   
                                                              2                   3.927962   
                                                              3                   4.318390   
                                                              4                   4.611590   
                                                              5                   4.928620   
                                                              6                   5.287702   
                                                              7                   5.276724   
                                                              8                   5.767487   
                                                              9                   7.736406   
                                                              10                  6.969109   
                                                              last tree           3.576387   
                       100                                    1                   3.708436   
                                                              2                   3.913780   
                                                              3           

In [18]:
# Show the ith tree height results for 20 tips (E[tMRCA_20] = 1.9).
height_df[height_df[r'Sampled Lineages $(n)$'] == 20].set_index([
    r'Sampled Lineages $(n)$',
    r'Population Recombination Rate $(\rho)$',
    r'$i^{th}$ Tree',
])

msprime Standard Coalescent  \
Sampled Lineages $(n)$ Population Recombination Rate $(\rho)$ $i^{th}$ Tree                                
20                     1                                      1                                 3.788942   
                                                              2                                 4.000020   
                                                              3                                 4.187951   
                                                              4                                 4.451812   
                                                              5                                 4.484654   
                                                              6                                 4.779287   
                                                              7                                 4.985304   
                                                              8                                 5.785205   
                                                              9                                 5.927770   
                                                              10                                4.646081   
                                                              last tree                         3.800070   
                       100                                    1                                 3.759887   
                                                              2                                 3.887631   
                                                              3                                 3.944989   
                                                              4                                 3.973936   
                                                              5                                 3.945427   
                                                              6                                 3.977242   
                                                              7                                 4.007448   
                                                              8                                 4.055541   
                                                              9                                 4.131580   
                                                              10                                4.076040   
                                                              last tree                         3.753499   

                                                                             msprime's SMC  \
Sampled Lineages $(n)$ Population Recombination Rate $(\rho)$ $i^{th}$ Tree                  
20                     1                                      1                   3.813122   
                                                              2                   4.024537   
                                                              3                   4.323497   
                                                              4                   4.545802   
                                                              5                   4.742379   
                                                              6                   4.779915   
                                                              7                   5.171782   
                                                              8                   5.392641   
                                                              9                   5.937667   
                                                              10                  6.620510   
                                                              last tree           3.776757   
                       100                                    1                   3.765883   
                                                              2                   3.948222   
                                                              3           

## `smclib.py` Code for Debugging

In [19]:
# Intialize a node class.
class Node:
    
    # Intialize the node.
    def __init__(self, node_id, age, node_type, parent=None, l_child=None, r_child=None):
        """
        Node Types
            - 0: leaf node
            - 1: coalescent event node
            - 2: visibile recombination
            - 3: hidden recombination
        """
        self.node_id = node_id
        self.age = age
        self.node_type = node_type
        self.parent = parent
        self.l_child = l_child
        self.r_child = r_child
        self.parent_dist = None
        self.l_child_dist = None
        self.r_child_dist = None
        
    # Define a deep copy method.
    def __deepcopy__(self, memo):
        """
        Return a deepy copy of an isntance of the Node class.
        """
        # Avoid infinite loops
        if id(self) in memo:
            return memo[id(self)]
        # Create a shallow copy of the current node
        copied_node = copy.copy(self)
        memo[id(self)] = copied_node
        # Deep copy children and parent
        copied_node.parent = copy.deepcopy(self.parent, memo)
        copied_node.l_child = copy.deepcopy(self.l_child, memo)
        copied_node.r_child = copy.deepcopy(self.r_child, memo)
        return copied_node
    
    # Define a method to check if a node is a leaf.
    def is_leaf(self):
        """
        True if the node is a leaf, False otherwise.
        """
        return self.node_type == 0
    
    # Define a method to compute the distance to the children.
    def dist_to_children(self):
        """
        Compute the distance from the current node to its children.
        """
        if self.l_child is not None:
            self.l_child_dist = self.age - self.l_child.age
        if self.r_child is not None:
            self.r_child_dist = self.age - self.r_child.age
    
    # Define a method to compute the distance to the parent node.
    def dist_to_parent(self):
        """
        Compute the distance from the current node to its parent.
        """
        if self.parent is not None:
            self.parent_dist = self.parent.age - self.age
            
    # Define a function to initialize distance to parent and children nodes.
    def init_dists(self):
        """
        Intialize the distances to the parent and children nodes.
        """
        self.dist_to_parent()
        self.dist_to_children()


# Intialize a tree class.
class Tree:
    
    # Intialize the tree.
    def __init__(self, left=0.0, right=1.0):
        self.left = left
        self.right = right
        self.root = None
        self.length = None
        self.next_node_id = None
        self.next_rec_id = -1
        self.nodes = {}
        self.edges = {}
        self.upper_bounds = None
        self.recomb_node = None
        self.coal_node = None
        
    def __deepcopy__(self, memo):
        """
        Return a deepy copy of an isntance of the Tree class.
        """
        # Avoid infinite loops.
        if id(self) in memo:
            return memo[id(self)]
        # Create a shallow copy of the tree.
        copied_tree    = copy.copy(self)
        memo[id(self)] = copied_tree
        # Deep copy nodes and edges.
        copied_tree.nodes = copy.deepcopy(self.nodes, memo)
        copied_tree.edges = copy.deepcopy(self.edges, memo)
        # Deep copy unary nodes.
        copied_tree.recomb_node = copy.deepcopy(self.recomb_node, memo)
        copied_tree.coal_node   = copy.deepcopy(self.coal_node, memo)
        return copied_tree
        
    # Define a method to add a node to the tree.
    def add_node(self, node):
        """
        Add a new node to the tree.
        """
        self.nodes[node.node_id] = node
        
    # Define a method to remove a node from the tree.
    def rmv_node(self, node):
        """
        Remove a new node to the tree.
        """
        del self.nodes[node.node_id]
    
    # Define a method to intialize node distances.
    def init_branch_lengths(self):
        """
        Intialize all the branch lengths for the current tree.
        """    
        # For every node.
        for node_id in self.nodes:
            # Intialize branch lengths.
            self.nodes[node_id].init_dists()
        
    # Define a method to intialize the edges on a tree.
    def init_edges(self):
        """
        Intialize all the edges on the current tree.
        """
        # Intialize variables.
        i = 0
        Lx = 0
        upper_bounds = []
        # For every node.
        for node in self.nodes:
            # If the node is not a leaf.
            if not self.nodes[node].is_leaf():
                # Record the interval's upper bound.
                upper_bounds.append(self.nodes[node].age)
                # Intialize the edge for parent -> left child.
                self.edges[i] = {}
                self.edges[i]['parent'] = self.nodes[node].node_id
                self.edges[i]['child'] = self.nodes[node].l_child.node_id
                self.edges[i]['upper'] = self.nodes[node].age
                self.edges[i]['lower'] = self.nodes[node].l_child.age
                self.edges[i]['length'] = self.nodes[node].l_child_dist
                i += 1
                Lx += self.nodes[node].l_child_dist
                # Intialize the edge for parent -> right child.
                self.edges[i] = {}
                self.edges[i]['parent'] = self.nodes[node].node_id
                self.edges[i]['child'] = self.nodes[node].r_child.node_id
                self.edges[i]['upper'] = self.nodes[node].age
                self.edges[i]['lower'] = self.nodes[node].r_child.age
                self.edges[i]['length'] = self.nodes[node].r_child_dist
                i += 1
                Lx += self.nodes[node].r_child_dist
        # Set the tree properties.
        self.upper_bounds = sorted(upper_bounds)
        self.length = Lx
                
    # Define a method to find the root node
    def find_root(self):
        """
        Determine the root node on the current tree.
        """
        root_node = max(self.nodes, key=lambda k: self.nodes[k].age)
        self.root = root_node
        
    # Define a method to reconcile the recoal and associated nodes.
    def perform_spr(self, broken_node, inherited_node, lonely_node, below_node, recoal_node, root_node):
        """
        Perform an SPR operation on the current tree.
        """
        # Intialize the new root.
        new_root = None
    
        ## [1] The broken and below nodes are the root node. ##
        if root_node.node_id == broken_node.node_id and root_node.node_id == below_node.node_id:
            
            # Reconcile the recoal node. #
            # Point the recoal node to its children: {inherited node, lonely node}.
            recoal_node.l_child = inherited_node
            recoal_node.r_child = lonely_node
            # Point the children: {inherited node, lonely node} to the recoal node.
            inherited_node.parent = recoal_node
            lonely_node.parent    = recoal_node
            # Set recoal node as the new root.
            new_root = recoal_node
        
        
        ## [2] The broken node is the below node and the root node is unique. ##
        elif broken_node.node_id == below_node.node_id and broken_node.node_id != root_node.node_id:

            # Reconcile the recoal node's parent node. #
            # Set the parent of the coal node as the parent of the recoal node.
            recoal_node.parent = self.coal_node.parent
            # If the left child of the parent is the broken/below node.
            if recoal_node.parent.l_child.node_id == below_node.node_id:
                # Replace the below node with the recoal node in the children set.
                recoal_node.parent.l_child = recoal_node
            # Else the right child of the parent is the below node.
            else:
                # Replace the below node with the recoal node in the children set.
                recoal_node.parent.r_child = recoal_node
            
            # Reconcile the recoal node. #
            # Point the recoal node to its children: {inherited node, lonely node}.
            recoal_node.l_child = inherited_node
            recoal_node.r_child = lonely_node
            # Point the children: {inherited node, lonely node} to the recoal node.
            inherited_node.parent = recoal_node
            lonely_node.parent    = recoal_node
        
        
        ## [3] The broken node is the root node and the below node is the lonely node. ##
        elif broken_node.node_id == root_node.node_id and below_node.node_id == lonely_node.node_id:

            # Reconcile the recoal node. #
            # Point the recoal node to its children: {inherited node, below node}.
            recoal_node.l_child = inherited_node
            recoal_node.r_child = below_node
            # Point the children: {inherited node, below node} to the recoal node.
            inherited_node.parent = recoal_node
            below_node.parent     = recoal_node
            # Set recoal node as the new root.
            new_root = recoal_node
        
        
        ## [4] The broken node is the root node and the below and lonely nodes are unique. ##
        elif broken_node.node_id == root_node.node_id and below_node.node_id != lonely_node.node_id:

            # Reconcile the recoal node's parent node. #
            # Set the parent of the coal node as the parent of the recoal node.
            recoal_node.parent = self.coal_node.parent
            # If the left child of the parent is the below node.
            if recoal_node.parent.l_child.node_id == below_node.node_id:
                # Replace the below node with the recoal node in the children set.
                recoal_node.parent.l_child = recoal_node
            # Else the right child of the parent is the below node.
            else:
                # Replace the below node with the recoal node in the children set.
                recoal_node.parent.r_child = recoal_node
            
            # Reconcile the recoal node. #
            # Point the recoal node to its children: {inherited node, below node}.
            recoal_node.l_child = inherited_node
            recoal_node.r_child = below_node
            # Point the children: {inherited node, below node} to the recoal node.
            inherited_node.parent = recoal_node
            below_node.parent     = recoal_node
            
            # Reconcile the lonely node. #
            # Set the lonely node as the new root.
            lonely_node.parent = None
            new_root = lonely_node
        
        
        ## [5] The below node is the root node and the broken node is unique. ##
        elif below_node.node_id == root_node.node_id and broken_node.node_id != root_node.node_id:
            
            # Reconcile the lonely node. #
            # Set the parent of the broken node as the new parent of the lonely node.
            lonely_node.parent = broken_node.parent
            # If the left child of the parent is the broken node.
            if lonely_node.parent.l_child.node_id == broken_node.node_id:
                # Replace the broken node with the lonely node in the children set.
                lonely_node.parent.l_child = lonely_node
            # Else the right child of is the broken node.
            else:
                # Replace the broken node with the lonely node in the children set.
                lonely_node.parent.r_child = lonely_node
            
            
            # Reconcile the recoal node. #
            # Point the recoal node to its children: {inherited node, below node}.
            recoal_node.l_child = inherited_node
            recoal_node.r_child = below_node
            # Point the children: {inherited node, below node} to the recoal node.
            inherited_node.parent = recoal_node
            below_node.parent     = recoal_node
            # Set recoal node as the new root.
            new_root = recoal_node
            
          
        ## [6] The broken node, below node, and root node are all unique, and the below node is the lonely node. ##
        elif broken_node.node_id != root_node.node_id and below_node.node_id == lonely_node.node_id:
            
            # Reconcile the recoal node's parent node. #
            # Set the parent of the broken node as the parent of the recoal node.
            recoal_node.parent = broken_node.parent
            # If the left child of the parent is the broken node.
            if recoal_node.parent.l_child.node_id == broken_node.node_id:
                # Replace the broken node with the recoal node in the children set.
                recoal_node.parent.l_child = recoal_node
            # Else the right child of the parent is the broken node.
            else:
                # Replace the broken node with the recoal node in the children set.
                recoal_node.parent.r_child = recoal_node
            
            # Reconcile the recoal node. #
            # Point the recoal node to its children: {inherited node, below node}.
            recoal_node.l_child = inherited_node
            recoal_node.r_child = below_node
            # Point the children: {inherited node, below node} to the recoal node.
            inherited_node.parent = recoal_node
            below_node.parent     = recoal_node
            
        ## [7] All nodes are unique. ##
        else:
            
            # Reconcile the lonely node. #
            # Set the parent of the broken node as the parent of the lonely node.
            lonely_node.parent = broken_node.parent
            # If the left child of is the broken node.
            if lonely_node.parent.l_child.node_id == broken_node.node_id:
                # Replace the broken node with the lonely node in the children set.
                lonely_node.parent.l_child = lonely_node
            # Else the right child of the below node is the broken node.
            else:
                # Replace the broken node with the lonely node in the children set.
                lonely_node.parent.r_child = lonely_node
                
            # Reconcile the recoal node's parent node. #
            # Set the parent of the coal node as the parent of the recoal node.
            recoal_node.parent = self.coal_node.parent
            # If the left child of the parent is the below node.
            if recoal_node.parent.l_child.node_id == below_node.node_id:
                # Replace the below node with the recoal node in the children set.
                recoal_node.parent.l_child = recoal_node
            # Else the right child of the parent is the below node.
            else:
                # Replace the below node with the recoal node in the children set.
                recoal_node.parent.r_child = recoal_node
            
            # Reconcile the recoal node. #
            # Point the recoal node to its children: {inherited node, below node}.
            recoal_node.l_child = inherited_node
            recoal_node.r_child = below_node
            # Point the children: {inherited node, below node} to the recoal node.
            inherited_node.parent = recoal_node
            below_node.parent     = recoal_node
        
        
        ## Tie up the new tree ##
        # Prune the broken node from the tree.
        self.rmv_node(broken_node)
        # Graft the recoal node to the tree.
        self.add_node(recoal_node)
        # Intialize branch lengths for the new tree.
        self.init_branch_lengths()
        # Intialize the edges for the new tree.
        self.init_edges()
        # Intialize the next node ids for the new tree.
        self.init_next_node_id()
        # If we already know the new root node.
        if new_root is not None:
            # Intialize the new root.
            self.root = new_root.node_id
        # Else, we do not know the root.
        else:
            # Intialize the new root.
            self.find_root()

    
    # Define a function to set the next node id.
    def init_next_node_id(self):
        """
        Set the next node id.
        """
        last_coal = self.coal_node
        max_node = max(self.nodes)
        if last_coal is not None:
            self.next_node_id = max([last_coal.node_id, max_node]) + 1
        else:
            self.next_node_id = max_node + 1
    
    # Define a method to recursively construct the newick information.
    def _to_newick_recursive(self, node):
        """
        Recursively construct the Newick information for a given node.
        """
        # Return the leaf id if the node is a leaf.
        if node.is_leaf():
            return f'{node.node_id}'
        # For internal nodes, get the newick information for each child.
        l_child_info = self._to_newick_recursive(node.l_child)
        r_child_info = self._to_newick_recursive(node.r_child)
        return '({}:{},{}:{})'.format(l_child_info, node.l_child_dist, 
                                      r_child_info, node.r_child_dist)
    
    # Define a method to export a tree in newick format.
    def to_newick(self):
        """
        Convert the tree to its Newick format.
        """
        # Start the conversion from the root.
        nwk = self._to_newick_recursive(self.nodes[self.root])
        # The Newick format ends with a semicolon.
        return nwk + ';'

In [20]:
# Define a function to intialize a tree from a msprime simulaion.
def init_msp_tree(k, Ne, ploidy, seed=None):
    """
    Returns a Tree object from a msprime simulation.
    
    k      -- Number of chromosomes to simulate.
    Ne     -- Effective population size.
    ploidy -- Haploid or diploid coalescent units.
    seed   -- Random seed for reporducibility.
    """
    # Simulate a tree under the standard coalescent.
    ts = msprime.sim_ancestry(
        samples=[msprime.SampleSet(k, ploidy=1)],
        population_size=Ne,
        ploidy=ploidy,
        random_seed=seed,
        discrete_genome=False,
    )
    # Intialize the current tree.
    tree = Tree()
    # For ever node.
    for node_id, age in enumerate(ts.tables.nodes.time):
        # If the node is a leaf.
        if age == 0:
            # Intialize the node.
            node = Node(
                node_id=node_id, age=(age / (Ne * ploidy)), node_type=0,
                parent=None, l_child=None, r_child=None,
            )
            # Add the node to the tree.
            tree.add_node(node)
        # Else, the node is an ancestral node.
        else:
            # Intialize the node.
            node = Node(
                node_id=node_id, age=(age / (Ne * ploidy)), node_type=1,
                parent=None, l_child=None, r_child=None,
            )
            # Add the node to the tree.
            tree.add_node(node)
    # For every parent node.
    for parent in np.unique(ts.tables.edges.parent):
        # Find the children of the parent node.
        left_child, right_child = ts.tables.edges[ts.tables.edges.parent == parent].child
        # Update the parent node for the two children.
        tree.nodes[left_child].parent = tree.nodes[parent]
        tree.nodes[right_child].parent = tree.nodes[parent]
        # Update the children nodes for the parent.
        tree.nodes[parent].l_child = tree.nodes[left_child]
        tree.nodes[parent].r_child = tree.nodes[right_child]
    # Intialize branch lengths.
    tree.init_branch_lengths()
    # Intialize the edges for the current tree.
    tree.init_edges()
    # Intialize the root node.
    tree.find_root()
    # Intialize the next node id.
    tree.init_next_node_id()
    # Make a copy of the tree-seq tables for editting.
    ts_tables = ts.dump_tables()
    return tree, ts, ts_tables

# Define a function to determine the distance to the next recombination event.
def draw_y(rho, Lx, ploidy):
    """
    Returns the distance to the next recombination event.
    
    rho    -- Population recombination rate.
    Lx     -- Total branch length of T_{x}.
    ploidy -- Haploid or diploid coalescent units.
    """
    print(f'total branch length: {Lx}')
    print(f'breakpoint scale paramter (2 / (rho * Lx)): {(2/ (rho * Lx))}')
    
    # Draw y.
    y = np.random.exponential((2 / (rho * Lx)))
    return y

# Define a function to determine the the lineage and age of the recombination event.
def draw_g(tree):
    """
    Returns the recombination event information for the current tree.
    
    tree -- An instance of the current tree.
    """
    # Compute the edge weights (ie edge_length/L(x)).
    edge_weights = [(tree.edges[key]['length'] / tree.length) for key in tree.edges.keys()]
    # Determine which edge will have the recombinatin event.
    rec_edge_key = np.random.choice(list(tree.edges.keys()), p=edge_weights)
    # Determine the age of the recombination event.
    g = np.random.uniform(tree.edges[rec_edge_key]['lower'], tree.edges[rec_edge_key]['upper'])
    return rec_edge_key, g

# Define a function to determine the lineage and age of the next coalescent event for the smc model.
def draw_coal_smc(tree, rec_edge_key, g, Ne, ploidy):
    """
    Returns the edge and coalescent information for the next tree.
    
    tree              -- An instance of the current tree
    rec_edge_key      -- Key of the edge with the recombination event in tree.edges.
    g                 -- Age of the recombination event on the current tree.
    Ne                -- Effective population size.
    ploidy            -- Haploid or diploid coalescent units.
    """
    # Intialize the lower bound of the first interval.
    c_lower_bound = 0
    # Intialize the key of the edge where the coalescent event will occur.
    coal_edge_key = None
    # For every possible coalescent interval.
    for i, c_upper_bound in enumerate(tree.upper_bounds):

        print(f'trying to find the re-coalesences event in the interval: {c_lower_bound} - {c_upper_bound}')

        # Determine if the recombination event occurs below the upper bound of the current interval.
        if c_upper_bound > g:
            # Determine the avaiable lineages in this interval.
            available_lineages = [
                key for key in tree.edges.keys() if\
                ((tree.edges[key]['upper'] >= c_upper_bound)\
                & (tree.edges[key]['lower'] <= c_lower_bound)\
                & (key != rec_edge_key))
            ]

            print(f'lineages in the interval ({c_lower_bound} - {c_upper_bound}): {available_lineages}')

            # If there are avaiable lineages.
            if len(available_lineages) > 0:

                print(f'there are {len(available_lineages)} lineages avaiable in the interval ({c_lower_bound} - {c_upper_bound})')
                print(f'the rate of coalesnces is {(1 / len(available_lineages))} during the interval ({c_lower_bound} - {c_upper_bound})')
                
                # Determine the time of the coalescent event.
                coal_time = g + np.random.exponential((1 / len(available_lineages)))
            # Else set the coalescent event to a variable that will fail.
            else:
                coal_time = -1
            # If the the coalescent event occurs within the current time interval.
            if c_upper_bound > coal_time > c_lower_bound:
                # Determine which edge the coalescent event occurs on.
                coal_edge_key = np.random.choice(available_lineages)

                print(f're-coalesences event occurs at {coal_time} on edge {coal_edge_key} during the interval ({c_lower_bound} - {c_upper_bound})')
                
                break
            # Else, re-intialize the lower bound and move on to the next interval.
            else:
                c_lower_bound = c_upper_bound
        # Else, re-intialize the lower bound and move on to the next interval.
        else:
            c_lower_bound = c_upper_bound
    # If an edge was not found within the current tree's interval.
    if coal_edge_key == None:
        # Determine the new time of coalescences above the root.
        coal_time = tree.nodes[tree.root].age + np.random.exponential(1)

        print(f're-coalesences event occurs above the current root at {coal_time})')

    return coal_edge_key, coal_time

# Define a function to determine the lineage and age of the next coalescent event for the smc' model.
def draw_coal_smc_prime(tree, g, Ne, ploidy):
    """
    Returns the edge and coalescent information for the next tree.
    
    tree   -- An instance of the current tree
    g      -- Age of the recombination event on the current tree.
    Ne     -- Effective population size.
    ploidy -- Haploid or diploid coalescent units.
    """
    # Intialize the lower bound of the first interval.
    c_lower_bound = 0
    # Intialize the key of the edge where the coalescent event will occur.
    coal_edge_key = None
    # For every possible coalescent interval.
    for i, c_upper_bound in enumerate(tree.upper_bounds):

        print(f'trying to find the re-coalesences event in the interval: {c_lower_bound} - {c_upper_bound}')
        
        # Determine if the recombination event occurs below the upper bound of the current interval.
        if c_upper_bound > g:
            # Determine the avaiable lineages in this interval.
            available_lineages = [
                key for key in tree.edges.keys() if\
                ((tree.edges[key]['upper'] >= c_upper_bound) & (tree.edges[key]['lower'] <= c_lower_bound))
            ] ### YOU CAN ADD THE CONDITION (key != rec_edge_key) FOR SMC ###

            print(f'lineages in the interval ({c_lower_bound} - {c_upper_bound}): {available_lineages}')
            
            # If there are avaiable lineages.
            if len(available_lineages) > 0:

                print(f'there are {len(available_lineages)} lineages avaiable in the interval ({c_lower_bound} - {c_upper_bound})')
                print(f'the rate of coalesnces is {(1 / len(available_lineages))} during the interval ({c_lower_bound} - {c_upper_bound})')
                
                # Determine the time of the coalescent event.
                coal_time = g + np.random.exponential((1 / len(available_lineages)))
            # Else set the coalescent event to a variable that will fail.
            else:
                coal_time = -1
            # If the the coalescent event occurs within the current time interval.
            if c_upper_bound > coal_time > c_lower_bound:
                # Determine which edge the coalescent event occurs on.
                coal_edge_key = np.random.choice(available_lineages)

                print(f're-coalesences event occurs at {coal_time} on edge {coal_edge_key} during the interval ({c_lower_bound} - {c_upper_bound})')
                
                break
            # Else, re-intialize the lower bound and move on to the next interval.
            else:
                c_lower_bound = c_upper_bound
        # Else, re-intialize the lower bound and move on to the next interval.
        else:
            c_lower_bound = c_upper_bound
    # If an edge was not found within the current tree's interval.
    if coal_edge_key == None:
        # Determine the new time of coalescences above the root.
        coal_time = tree.nodes[tree.root].age + np.random.exponential(1)

        print(f're-coalesences event occurs above the current root at {coal_time})')

    return coal_edge_key, coal_time

__Note:__ the code below is simulating under the SMC' algorithim with the SMC print out for demonstrative purposes only.

In [21]:
## (0) Intialize the inputs for the the SMC/SMC' algorithim. ###
k = 3
Ne = 1
rho = 1
ploidy = 2
seed = None

## (1) Intialize the first tree, T(x)=T_{0}, at position x=0, and compute the total branch length L(x)=L_{0}. ##

# Intialize a tree-sequence dictionary.
ts_dicc = {}
# Intialize the first tree index.
tree_idx = 0
# Intialize the start position.
x = 0
# Simulate a tree (T_{0}) under the standard coalescent at point x=0.
c_tree, ts, ts_tables = init_msp_tree(k=k, Ne=Ne, ploidy=ploidy, seed=seed)

print(ts.draw_text())

## (2) Generate the distance, y=exp[(rho/2)L(x)], to the next recombination event. ##

# Compute the distance to the next recombination event (y).
y = draw_y(rho=rho, Lx=c_tree.length, ploidy=ploidy)

print(f'first interval: [{x}, {y})')

# While we are still within the sequence intervals.
while (x + y) < 1:
    # Intialize the new right position
    c_tree.right = (x + y)
    
## (3) Determine the location (ie what edge), and the age of the recombination event (g). ##

    # Determine g and its location on the current tree.
    rec_edge_key, g = draw_g(tree=c_tree)
        
    print(f'recombination event age: {g}')
    print('recombination edge:')
    print(c_tree.edges[rec_edge_key])
    print('\n')
    
## (4) Overlay the recombination event at time g and allow the branch below g to coalesce elsewhere on the tree. ##

    print('--------------------------------------------------------------------------------')
    print('***SMC DRAW RECOALESCENCES***')
    print('\n')

    # Deteremine the location and time of the recombining coalescent event.
    coal_edge_key, coal_time = draw_coal_smc(tree=c_tree, rec_edge_key=rec_edge_key, g=g, Ne=Ne, ploidy=ploidy)

    print(f'recombination event age: {g}; re-coalesence age: {coal_time}')
    if coal_edge_key is None:
        print(f're-coalesence is above the root node (ie, node id: {c_tree.root})')
    else:
        print('re-coalesence edge:')
        print(c_tree.edges[coal_edge_key])
    print('\n')

    print('--------------------------------------------------------------------------------')
    print('***SMC\' DRAW RECOALESCENCES***')
    print('\n')
    
    # Deteremine the location and time of the recombining coalescent event.
    coal_edge_key, coal_time = draw_coal_smc_prime(tree=c_tree, g=g, Ne=Ne, ploidy=ploidy)
    
    print(f'recombination event age: {g}; re-coalesence age: {coal_time}')
    if coal_edge_key is None:
        print(f're-coalesence is above the root node (ie, node id: {c_tree.root})')
    else:
        print('re-coalesence edge:')
        print(c_tree.edges[coal_edge_key])
    print('--------------------------------------------------------------------------------')
    print('\n')
        
## (5) Prune the old branch above g and graft the new branch to construct the next tree at position x+y. ##
    
    ### HIDDEN RECOMBINATION SCENARIO ###
    
    # If the coalescent event is hidden (ie recombination and coalesence occur on the same branch).
    if rec_edge_key == coal_edge_key:
        
        print('hidden recombination event')
        
        # Intialize a recombination event node for the current tree.
        # Note this is purely for bookkeeping purposes for future conversion to a gARG.
        recomb_node = Node(
            node_id=c_tree.next_rec_id, age=g, node_type=3,
            parent=c_tree.nodes[c_tree.edges[rec_edge_key]['parent']],
            l_child=c_tree.nodes[c_tree.edges[rec_edge_key]['child']], r_child=None,
        )
        # Intialize the coal node for the current tree.
        # Note this is purely for bookkeeping purposes for future conversion to a gARG.
        coal_node = Node(
            node_id=c_tree.next_node_id, age=coal_time, node_type=1,
            parent=c_tree.nodes[c_tree.edges[coal_edge_key]['parent']],
            l_child=c_tree.nodes[c_tree.edges[coal_edge_key]['child']], r_child=None,
        )
        # Move the the recombination event node counter back.
        c_tree.next_rec_id -= 1
        # Move the coalescent event node counter forward.
        c_tree.next_node_id += 1
        # Record the recombination event.
        c_tree.recomb_node = recomb_node
        # Record the re-coalesence event.
        c_tree.coal_node = coal_node
        # Add the current tree to the tree-sequence dictionary.
        ts_dicc[tree_idx] = c_tree
        # Move the tree index forward.
        tree_idx += 1
        # Intialize the next tree by copying the current tree.
        n_tree       = copy.deepcopy(c_tree)
        n_tree.left  = (x + y)
        n_tree.right = 1.0
    # Else, the coalescent event is not hidden.
    else:
        
        ### Perform the SPR Algorithim ###
        
        # Intialize unary nodes—ie the recomb and coal nodes. #
        
        # Intialize a recombination event node for the current tree.
        # Note this is purely for bookkeeping purposes for future conversion to a gARG.
        recomb_node = Node(
            node_id=c_tree.next_rec_id, age=g, node_type=2,
            parent=c_tree.nodes[c_tree.edges[rec_edge_key]['parent']],
            l_child=c_tree.nodes[c_tree.edges[rec_edge_key]['child']], r_child=None,
        )
        # Move the the recombination event node counter back.
        c_tree.next_rec_id -= 1
        # Record the recombination event.
        c_tree.recomb_node = recomb_node
        # If the re-coalescence event is above the root.
        if coal_edge_key == None:
            # Intialize the next node id.
            next_id = c_tree.next_node_id
            # Intialize the coal node for the current tree.
            # Note this is purely for bookkeeping purposes for future conversion to a gARG.
            coal_node = Node(
                node_id=next_id, age=coal_time, node_type=1,
                parent=None, l_child=c_tree.nodes[c_tree.root], r_child=None,
            )
            # Record the coal node on the current tree.
            c_tree.coal_node = coal_node
            # Add the current tree to the tree-sequence dictionary.
            ts_dicc[tree_idx] = c_tree
            # Move the tree index forward.
            tree_idx += 1
            # Intialize the next tree by copying the current tree.
            n_tree       = copy.deepcopy(c_tree)
            n_tree.left  = (x + y)
            n_tree.right = 1.0
        # Else the re-coalescence event is on an existing edge.
        else:
            # Intialize the next node id.
            next_id = c_tree.next_node_id
            # Intialize the coal node for the current tree.
            # Note this is purely for bookkeeping purposes for future conversion to a gARG.
            coal_node = Node(
                node_id=next_id, age=coal_time, node_type=1,
                parent=c_tree.nodes[c_tree.edges[coal_edge_key]['parent']],
                l_child=c_tree.nodes[c_tree.edges[coal_edge_key]['child']], r_child=None,
            )
            # Record the coal node on the current tree.
            c_tree.coal_node = coal_node
            # Add the current tree to the tree-sequence dictionary.
            ts_dicc[tree_idx] = c_tree
            # Move the tree index forward.
            tree_idx += 1
            # Intialize the next tree by copying the current tree.
            n_tree       = copy.deepcopy(c_tree)
            n_tree.left  = (x + y)
            n_tree.right = 1.0
        
        # Intialize nodes of interest. #
        broken_node    = n_tree.recomb_node.parent
        inherited_node = n_tree.recomb_node.l_child
        lonely_node    = broken_node.r_child if broken_node.l_child == inherited_node else broken_node.l_child
        below_node     = n_tree.coal_node.l_child
        root_node      = n_tree.nodes[n_tree.root]
        # Intialize the re-coalesence event node to create the next tree.
        recoal_node = Node(
            node_id=next_id, age=coal_time, node_type=1,
            parent=None, l_child=None, r_child=None,
        )
        
        print(f'broken node: {broken_node.node_id}')
        print(f'inherited node: {inherited_node.node_id}')
        print(f'lonely node {lonely_node.node_id}')
        print(f'below node: {below_node.node_id}')
        print(f'root node: {root_node.node_id}')
        
        ## [1] The broken and below nodes are the root node. ##
        if root_node.node_id == broken_node.node_id and root_node.node_id == below_node.node_id:
            
            print('spr scenario 1')
            
        ## [2] The broken node is the below node and the root node is unique. ##
        elif broken_node.node_id == below_node.node_id and broken_node.node_id != root_node.node_id:
            
            print('spr scenario 2')
        
        ## [3] The broken node is the root node and the below node is the lonely node. ##
        elif broken_node.node_id == root_node.node_id and below_node.node_id == lonely_node.node_id:
            
            print('spr scenario 3')
        
        ## [4] The broken node is the root node and the below and lonely nodes are unique. ##
        elif broken_node.node_id == root_node.node_id and below_node.node_id != lonely_node.node_id:
            
            print('spr scenario 4')
        
         ## [5] The below node is the root node and the broken node is unique. ##
        elif below_node.node_id == root_node.node_id and broken_node.node_id != root_node.node_id:
            
            print('spr scenario 5')
        
        ## [6] The broken node, below node, and root node are all unique, and the below node is the lonely node. ##
        elif broken_node.node_id != root_node.node_id and below_node.node_id == lonely_node.node_id:
            
            print('spr scenario 6')
        
        ## [7] All nodes are unique. ##
        else:
            
            print('spr scenario 7')
        
        ## Reconcile the new tree! ##
        n_tree.perform_spr(
            broken_node=broken_node, inherited_node=inherited_node, lonely_node=lonely_node,
            below_node=below_node, recoal_node=recoal_node, root_node=root_node,
        )

        
    # Set the new tree as the current tree.
    c_tree = n_tree
        
## (6) Reset the new interval x=x+y, intialize the new tree as the current tree T(x), and compute the compute the total branch length L(x). ##
    
    # Reset the new left interval (x).
    x = (x + y)
    # Compute the distance to the next recombination event (y).
    y = draw_y(rho=rho, Lx=c_tree.length, ploidy=ploidy)
    
    print(f'distance to the next brakepoint: {y}')
    print('--------------------------------------------------------------------------------')
    print('\n')
    print(f'next interval: [{x}, {x + y})')

# Remove the recombination an re-coalescence nodes from the last tree,
# that did not expirence recombination.
c_tree.recomb_node = None
c_tree.recoal_node = None

print(f'last interval: [{c_tree.left}, {c_tree.right})')

# Add the last tree to the tree-sequence.
ts_dicc[tree_idx] = c_tree

2.57┊   4   ┊
    ┊  ┏┻━┓ ┊
0.23┊  3  ┃ ┊
    ┊ ┏┻┓ ┃ ┊
0.00┊ 0 2 1 ┊
    0       1

total branch length: 2.6815444793260035
breakpoint scale paramter (2 / (rho * Lx)): 0.7458388310988199
first interval: [0, 0.8341474031261266)
recombination event age: 0.5261370789049321
recombination edge:
{'parent': 4, 'child': 3, 'upper': 1.2825859822082317, 'lower': 0.11637251490954022, 'length': 1.1662134672986915}


--------------------------------------------------------------------------------
***SMC DRAW RECOALESCENCES***


trying to find the re-coalesences event in the interval: 0 - 0.11637251490954022
trying to find the re-coalesences event in the interval: 0.11637251490954022 - 1.2825859822082317
lineages in the interval (0.11637251490954022 - 1.2825859822082317): [2]
there are 1 lineages avaiable in the interval (0.11637251490954022 - 1.2825859822082317)
the rate of coalesnces is 1.0 during the interval (0.11637251490954022 - 1.2825859822082317)
re-coalesences event occurs at 0.64885200156

In [22]:
print(f'there are {len(ts_dicc)} trees in the tree-sequence')
print('\n')
for tree in ts_dicc:
    print(f'tree: {tree}')
    print(f'[left, right): [{ts_dicc[tree].left}, {ts_dicc[tree].right})')
    if (ts_dicc[tree].recomb_node is not None) & (ts_dicc[tree].coal_node is not None):
        recomb_info = []
        recomb_info.append(ts_dicc[tree].recomb_node.node_id)
        recomb_info.append(ts_dicc[tree].recomb_node.age)
        recomb_info.append(ts_dicc[tree].recomb_node.parent.node_id)
        recomb_info.append(ts_dicc[tree].recomb_node.l_child.node_id)
        coal_info = []
        coal_info.append(ts_dicc[tree].coal_node.node_id)
        coal_info.append(ts_dicc[tree].coal_node.age)
        if ts_dicc[tree].coal_node.parent is not None:
            coal_info.append(ts_dicc[tree].coal_node.parent.node_id)
        else:
            coal_info.append(ts_dicc[tree].root)
        coal_info.append(ts_dicc[tree].coal_node.l_child.node_id)
        print(f'recomb event id: {recomb_info[0]}, age: {recomb_info[1]}, parent: {recomb_info[2]}, child: {recomb_info[3]}')
        print(f'coal event id: {coal_info[0]}, age: {coal_info[1]}, parent: {coal_info[2]}, child: {coal_info[3]}')
    for node in ts_dicc[tree].nodes:
        if ts_dicc[tree].nodes[node].is_leaf():
            print(f'leaf: {ts_dicc[tree].nodes[node].node_id}, parent: {ts_dicc[tree].nodes[node].parent.node_id}')
        else:
            print(f'{ts_dicc[tree].nodes[node].node_id}: ({ts_dicc[tree].nodes[node].l_child.node_id}, {ts_dicc[tree].nodes[node].r_child.node_id})')
    for edge in ts_dicc[tree].edges:
        print(ts_dicc[tree].edges[edge])
    print('\n')

there are 2 trees in the tree-sequence


tree: 0
[left, right): [0.0, 0.8341474031261266)
recomb event id: -1, age: 0.5261370789049321, parent: 4, child: 3
coal event id: 5, age: 1.209727325902886, parent: 4, child: 1
leaf: 0, parent: 3
leaf: 1, parent: 4
leaf: 2, parent: 3
3: (0, 2)
4: (1, 3)
{'parent': 3, 'child': 0, 'upper': 0.11637251490954022, 'lower': 0.0, 'length': 0.11637251490954022}
{'parent': 3, 'child': 2, 'upper': 0.11637251490954022, 'lower': 0.0, 'length': 0.11637251490954022}
{'parent': 4, 'child': 1, 'upper': 1.2825859822082317, 'lower': 0.0, 'length': 1.2825859822082317}
{'parent': 4, 'child': 3, 'upper': 1.2825859822082317, 'lower': 0.11637251490954022, 'length': 1.1662134672986915}


tree: 1
[left, right): [0.8341474031261266, 1.0)
leaf: 0, parent: 3
leaf: 1, parent: 5
leaf: 2, parent: 3
3: (0, 2)
5: (3, 1)
{'parent': 3, 'child': 0, 'upper': 0.11637251490954022, 'lower': 0.0, 'length': 0.11637251490954022}
{'parent': 3, 'child': 2, 'upper': 0.11637251490954022, 'low